# Definable types as systems of equations

Our goal is to make pomagma.reducer smart enough to prove that `I:UNIT`, as in
definable_types.text (2016:08:23-25) (Q2) and
operational_semantics.text (2017:01:20-02:05) (Q5.A2.9):

<b>Desired Theorem:</b> `I : A \a,a',f,x. a(f(a' x))`, where

    copy := \x,y. x y y.
    join := \x,y,z. x(y|z).
    postconj := (\r,s. <B r, B s>).
    preconj := (\r,s. <CB s, CB r>).
    compose := (\r,s,r',s'. <r o r', s' o s>).
    A = A | <I, I> | <copy, join> | <div, BOT> | <BOT, TOP> | <C, C>
          | A preconj | A postconj | A (A compose).

In [1]:
from pomagma.reducer.lib import BOT, TOP, B, C, I, pair
from pomagma.reducer.sugar import as_term
from pomagma.reducer.syntax import NVAR, sexpr_print
from pomagma.reducer.systems import System, try_compute_step, try_decide_equal

We'll start with a definition of `A` by mutual recursion.

In [2]:
i = NVAR('i')
b = NVAR('b')
c = NVAR('c')
cb = NVAR('cb')
div = NVAR('div')
copy = NVAR('copy')
join = NVAR('join')
postconj = NVAR('postconj')
preconj = NVAR('preconj')
compose = NVAR('compose')
a = NVAR('a')

system_a = System(
    # Basic combinators.
    i=I,
    b=B,
    c=C,
    cb=C(B),
    # Components of A.
    div=(I | as_term(lambda x: div(x, TOP))),
    copy=as_term(lambda x, y: x(y, y)),
    join=as_term(lambda x, y, z: x(y | z)),
    postconj=as_term(lambda r, s: pair(b(r), b(s))),
    preconj=as_term(lambda r, s: pair(cb(s), cb(r))),
    compose=as_term(lambda r1, s1, r2, s2: pair(b(r1, r2), b(s2, s1))),
    # Definition of A, intended as a least fixed point.
    # We'll comment out most parts to focus on the others.
    a=(pair(i,i)
       # | pair(copy, join)
       # | pair(div, BOT)
       # | pair(c, c)
       | a(preconj)
       # | a(postconj)
       | a(a(compose))
    ),
)
print(system_a.pretty())

       a = (JOIN (a preconj) (JOIN (a (a compose)) (ABS (0 i i))))
       b = (ABS (ABS (ABS (2 (1 0)))))
       c = (ABS (ABS (ABS (2 0 1))))
      cb = (ABS (ABS (ABS (1 (2 0)))))
 compose = (ABS (ABS (ABS (ABS (ABS (0 (b 4 2) (b 1 3)))))))
    copy = (ABS (ABS (1 0 0)))
     div = (JOIN (ABS 0) (ABS (div 0 TOP)))
       i = (ABS 0)
    join = (ABS (ABS (ABS (2 (JOIN 0 1)))))
postconj = (ABS (ABS (ABS (0 (b 2) (b 1)))))
 preconj = (ABS (ABS (ABS (0 (cb 1) (cb 2)))))


Next we'll extend this system to define a `UNIT` type.

A safer implementation of the type constructor `Simple` would apply `V` and check the input signature (e.g. here `unit_sig`). To keep the system simple, we'll directly apply `A` to the signature.

In [3]:
unit_sig = NVAR('unit_sig')
unit = NVAR('unit')

system_unit = system_a.extended_by(
    unit_sig=as_term(lambda r, s, f, x: r(f(s(x)))),
    unit=a(unit_sig),
)
print(system_unit.pretty())

       a = (JOIN (a preconj) (JOIN (a (a compose)) (ABS (0 i i))))
       b = (ABS (ABS (ABS (2 (1 0)))))
       c = (ABS (ABS (ABS (2 0 1))))
      cb = (ABS (ABS (ABS (1 (2 0)))))
 compose = (ABS (ABS (ABS (ABS (ABS (0 (b 4 2) (b 1 3)))))))
    copy = (ABS (ABS (1 0 0)))
     div = (JOIN (ABS 0) (ABS (div 0 TOP)))
       i = (ABS 0)
    join = (ABS (ABS (ABS (2 (JOIN 0 1)))))
postconj = (ABS (ABS (ABS (0 (b 2) (b 1)))))
 preconj = (ABS (ABS (ABS (0 (cb 1) (cb 2)))))
unit_sig = (ABS (ABS (ABS (ABS (3 (1 (2 0)))))))
    unit = (a unit_sig)


## Type checking

Now we can try type checking

In [4]:
try:
    print(try_decide_equal(system_unit, TOP, unit(TOP)))
    print(try_decide_equal(system_unit, BOT, unit(BOT)))
    print(try_decide_equal(system_unit, i, unit(i)))
except NotImplementedError as e:
    print(e)

TODO handle JOIN: JOIN(APP(APP(APP(NVAR(a), NVAR(preconj)), NVAR(unit_sig)), TOP), JOIN(APP(APP(APP(NVAR(unit_sig), NVAR(i)), NVAR(i)), TOP), APP(APP(APP(NVAR(a), APP(NVAR(a), NVAR(compose))), NVAR(unit_sig)), TOP))) vs TOP


Since the decision procedure is difficult to implement, let's instead try reducing.

## Tracing reduction sequences

In [5]:
def trace(system, name, steps=10):
    '''Print a reduction sequence'''
    system = system.copy()
    print('0. {}'.format(sexpr_print(system[name])))
    for step in xrange(steps):
        if not try_compute_step(system, name):
            print '[ Normalized ]'
            return
        print('{}. {}'.format(1 + step, sexpr_print(system[name])))
    print('[ Not Normalized ]')

In [6]:
system_test = system_unit.extended_by(
    unit_i=unit(i),
    unit_bot=unit(BOT),
    unit_top=unit(TOP),
)

In [7]:
trace(system_test, 'unit_top', 10)

0. (unit TOP)
1. (a unit_sig TOP)
2. (JOIN (a preconj unit_sig TOP) (JOIN (unit_sig i i TOP) (a (a compose) unit_sig TOP)))
3. (JOIN (unit_sig i i TOP) (JOIN (a preconj preconj unit_sig TOP) (JOIN (preconj i i unit_sig TOP) (JOIN (a (a compose) unit_sig TOP) (a (a compose) preconj unit_sig TOP)))))
4. (JOIN (ABS (i TOP)) (JOIN (a preconj preconj unit_sig TOP) (JOIN (preconj i i unit_sig TOP) (JOIN (a (a compose) unit_sig TOP) (a (a compose) preconj unit_sig TOP)))))
5. TOP
[ Normalized ]


In [8]:
trace(system_test, 'unit_bot', 10)

0. (unit BOT)
1. (a unit_sig BOT)
2. (JOIN (a preconj unit_sig BOT) (JOIN (unit_sig i i BOT) (a (a compose) unit_sig BOT)))
3. (JOIN (unit_sig i i BOT) (JOIN (a preconj preconj unit_sig BOT) (JOIN (preconj i i unit_sig BOT) (JOIN (a (a compose) unit_sig BOT) (a (a compose) preconj unit_sig BOT)))))
4. (JOIN (ABS (i BOT)) (JOIN (a preconj preconj unit_sig BOT) (JOIN (preconj i i unit_sig BOT) (JOIN (a (a compose) unit_sig BOT) (a (a compose) preconj unit_sig BOT)))))
5. (JOIN (a preconj preconj unit_sig BOT) (JOIN (preconj i i unit_sig BOT) (JOIN (a (a compose) unit_sig BOT) (a (a compose) preconj unit_sig BOT))))
6. (JOIN (preconj i i unit_sig BOT) (JOIN (a (a compose) unit_sig BOT) (JOIN (a preconj preconj preconj unit_sig BOT) (JOIN (preconj i i preconj unit_sig BOT) (JOIN (a (a compose) preconj unit_sig BOT) (a (a compose) preconj preconj unit_sig BOT))))))
7. (JOIN (a (a compose) unit_sig BOT) (JOIN (unit_sig (cb i) (cb i) BOT) (JOIN (a preconj preconj preconj unit_sig BOT) (JOIN (

In [9]:
trace(system_test, 'unit_i', 10)

0. (unit i)
1. (a unit_sig i)
2. (JOIN (a preconj unit_sig i) (JOIN (unit_sig i i i) (a (a compose) unit_sig i)))
3. (JOIN (unit_sig i i i) (JOIN (a preconj preconj unit_sig i) (JOIN (preconj i i unit_sig i) (JOIN (a (a compose) unit_sig i) (a (a compose) preconj unit_sig i)))))
4. (JOIN (ABS (i (i (i 0)))) (JOIN (a preconj preconj unit_sig i) (JOIN (preconj i i unit_sig i) (JOIN (a (a compose) unit_sig i) (a (a compose) preconj unit_sig i)))))
5. (JOIN (ABS (i (i 0))) (JOIN (a preconj preconj unit_sig i) (JOIN (preconj i i unit_sig i) (JOIN (a (a compose) unit_sig i) (a (a compose) preconj unit_sig i)))))
6. (JOIN i (JOIN (a preconj preconj unit_sig i) (JOIN (preconj i i unit_sig i) (JOIN (a (a compose) unit_sig i) (a (a compose) preconj unit_sig i)))))
7. (JOIN (ABS 0) (JOIN (a preconj preconj unit_sig i) (JOIN (preconj i i unit_sig i) (JOIN (a (a compose) unit_sig i) (a (a compose) preconj unit_sig i)))))
8. (JOIN (ABS 0) (JOIN (preconj i i unit_sig i) (JOIN (a (a compose) unit_sig 